In [1]:
import pandas as pd
import os

In [2]:
#load events data
events_path = os.path.join('../data', 'events.csv')
events_usecols = ['direction', 'size', 'status', 'timestamp', 'client.user_id','time.backend']

events_data = pd.read_csv(events_path, usecols=events_usecols)
events_data = events_data.rename(columns={'client.user_id': 'client_user_id','time.backend': 'time_backend'})

In [4]:
# inspect the data
events_data.head()

,direction,size,status,timestamp,client_user_id,time_backend
0,upload,131407,success,1651292104294,b949f0af-f46a-4b89-aaab-48ad788bdea3,1803
1,upload,4508141,success,1651292105362,c7646e9f-6698-41dc-a3ee-90c8d9eb7feb,855
2,upload,89976,success,1651292107281,b949f0af-f46a-4b89-aaab-48ad788bdea3,2203
3,upload,131368,success,1651292108717,b949f0af-f46a-4b89-aaab-48ad788bdea3,1813
4,upload,100992,success,1651292111561,b949f0af-f46a-4b89-aaab-48ad788bdea3,2137


In [6]:
events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341399 entries, 0 to 341398
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   direction       341399 non-null  object
 1   size            341399 non-null  int64 
 2   status          341399 non-null  object
 3   timestamp       341399 non-null  int64 
 4   client_user_id  341399 non-null  object
 5   time_backend    341399 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 15.6+ MB


In [8]:
events_data.describe()

,size,timestamp,time_backend
count,3.413990e+05,3.413990e+05,3.413990e+05
mean,1.033448e+06,1.651401e+12,5.177738e+04
std,2.750060e+06,7.367239e+07,1.470133e+07
min,1.000000e+00,1.651270e+12,1.080000e+02
25%,1.144850e+04,1.651330e+12,7.050000e+02
50%,9.388700e+04,1.651410e+12,1.049000e+03
75%,8.381815e+05,1.651476e+12,1.593000e+03
max,6.408020e+07,1.651514e+12,4.294967e+09


In [23]:
# load events data calling /ingest api
import json
import requests
for i in range(len(events_data)):
    event = json.loads(events_data.iloc[i].to_json())
    r = requests.post('http://127.0.0.1:8000/ingest/', data=event)
    if not r.status_code == requests.codes.created:
        print('error inserting event', i, 'event: ', event, 'response:', r)
    if i%1000 == 0:
        print("request n", i)

request n 0
request n 1000
request n 2000
request n 3000
request n 4000
request n 5000
request n 6000
request n 7000
request n 8000
request n 9000
request n 10000
request n 11000
request n 12000
request n 13000
request n 14000
request n 15000
request n 16000
request n 17000
request n 18000
request n 19000
request n 20000
request n 21000
request n 22000
request n 23000
request n 24000
request n 25000
request n 26000
request n 27000
request n 28000
request n 29000
request n 30000
request n 31000
request n 32000
request n 33000
request n 34000
request n 35000
request n 36000
request n 37000
request n 38000
request n 39000
request n 40000
request n 41000
request n 42000
request n 43000
request n 44000
request n 45000
request n 46000
request n 47000
request n 48000
request n 49000
request n 50000
request n 51000
request n 52000
request n 53000
request n 54000
request n 55000
request n 56000
request n 57000
request n 58000
request n 59000
request n 60000
request n 61000
request n 62000
reque

In [24]:
# create sqlite connection
from sqlite3 import connect
sqlite_path = '../cubbit_django/db.sqlite3'
conn_sqlite = connect(sqlite_path)

In [25]:
# read events data from sqlite
events_data_sqlite = pd.read_sql('select * from data_ingestion_event', conn_sqlite)

In [26]:
# inspect the data
events_data_sqlite.head()

,id,client_user_id,timestamp,size,time_backend,status,direction
0,125828,b949f0af-f46a-4b89-aaab-48ad788bdea3,1651292104294,131407,1803,success,upload
1,125829,c7646e9f-6698-41dc-a3ee-90c8d9eb7feb,1651292105362,4508141,855,success,upload
2,125830,b949f0af-f46a-4b89-aaab-48ad788bdea3,1651292107281,89976,2203,success,upload
3,125831,b949f0af-f46a-4b89-aaab-48ad788bdea3,1651292108717,131368,1813,success,upload
4,125832,b949f0af-f46a-4b89-aaab-48ad788bdea3,1651292111561,100992,2137,success,upload


In [27]:
# check that all rows have been inserted
query_count_res = pd.read_sql('select COUNT(*) as count from data_ingestion_event', conn_sqlite)
inserted_rows = query_count_res['count'].iloc[0]
rows_to_be_inserted = len(events_data)
inserted_rows, rows_to_be_inserted 

(341399, 341399)